In [ ]:
# Creating validation dataset
import os, random, shutil
import os.path

source = r'/content/drive/MyDrive/Fall 2022/ECS271/detecto/Sohas_weapon-Detection/train'
dest = r'/content/drive/MyDrive/Fall 2022/ECS271/detecto/Sohas_weapon-Detection/validation'
no_of_files = 500

print("%"*25+"{ Details Of Transfer }"+"%"*25)
print("\n\nList of Files Moved to %s :-"%(dest))

count = 0
files_list = []

while count < no_of_files:
    random_file = random.choice(os.listdir(source))
    
    if '(' not in random_file:
      if not os.path.exists("%s/%s"%(source, random_file.split('.')[0]+".xml")):
        os.remove(os.path.join(source, random_file))
      
      if (random_file.split('.')[1] != "xml" and count == 0) or (random_file.split('.')[1] != "xml" and (random_file not in files_list)):
        count += 1
        files_list.append(random_file)
        print("%d} %s"%(count, random_file))
        source_file = os.path.join(source, random_file)
        source_xml_file = os.path.join(source, random_file.split('.')[0]+".xml")
        dest_file = dest
        shutil.move(source_file, dest_file)
        shutil.move(source_xml_file, dest_file)

print("\n\n"+"$"*33+"[ Files Moved Successfully ]"+"$"*33)

In [ ]:
import torch

In [ ]:
!pip install detecto

In [ ]:
from detecto import core, utils, visualize
from detecto.visualize import show_labeled_image, plot_prediction_grid
from torchvision import transforms
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
train_dataset = core.Dataset('/content/drive/MyDrive/Fall 2022/ECS271/detecto/Sohas_weapon-Detection/train/')
val_dataset = core.Dataset('/content/drive/MyDrive/Fall 2022/ECS271/detecto/Sohas_weapon-Detection/validation/')
loader = core.DataLoader(train_dataset, batch_size=4, shuffle=True)
model = core.Model(['pistol', 'smartphone', 'knife', 'monedero', 'billete', 'tarjeta'])
losses = model.fit(loader, val_dataset, epochs=1, lr_step_size=5, learning_rate=0.001, verbose=True)

plt.plot(losses)
plt.show()

model.save('detecto_model_weights.pth')

In [ ]:
model = core.Model.load('/content/drive/MyDrive/Fall 2022/ECS271/detecto/detecto_model_weights.pth', ['pistol', 'smartphone', 'knife', 'monedero', 'billete', 'tarjeta'])

In [ ]:
image = utils.read_image('/content/drive/MyDrive/Fall 2022/ECS271/detecto/Sohas_weapon-Detection/test/HBsframe00235.jpg') 
predictions = model.predict(image)
labels, boxes, scores = predictions
show_labeled_image(image, boxes, labels)

In [ ]:
image = utils.read_image('/content/drive/MyDrive/Fall 2022/ECS271/detecto/Sohas_weapon-Detection/test/HBsframe00235.jpg') 
predictions = model.predict(image)
labels, boxes, scores = predictions

thresh = 0.6
filtered_indices = np.where(scores>thresh)
filtered_scores = scores[filtered_indices]
filtered_boxes = boxes[filtered_indices]
num_list = filtered_indices[0].tolist()
filtered_labels = [labels[i] for i in num_list]
show_labeled_image(image, filtered_boxes, filtered_labels)

In [ ]:
import cv2
from google.colab.patches import cv2_imshow

image = utils.read_image('/content/drive/MyDrive/Fall 2022/ECS271/detecto/Sohas_weapon-Detection/test/KravMagaKnifeDefenseTechniques153.jpg') 
predictions = model.predict(image)
labels, boxes, scores = predictions

img = cv2.imread("/content/drive/MyDrive/Fall 2022/ECS271/detecto/Sohas_weapon-Detection/test/KravMagaKnifeDefenseTechniques153.jpg")
print("Original image")
cv2_imshow(img)
x = int(boxes[np.argmax(scores)][0])
y = int(boxes[np.argmax(scores)][1])
w = int(boxes[np.argmax(scores)][2])
h = int(boxes[np.argmax(scores)][3])
crop_img = img[y:h, x:w]
print("Detected object")
cv2_imshow(crop_img)

In [ ]:
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
import torch
import torchvision
import torchvision.transforms as transforms
import seaborn as sns
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(3, 96, 11, stride=2)
    self.pool1 = nn.MaxPool2d(4, stride=2)
    self.conv2 = nn.Conv2d(96, 128, 3, stride=2, padding=2)
    self.pool2 = nn.MaxPool2d(4, stride=2)
    self.conv3 = nn.Conv2d(128, 256, 3, stride=1, padding=1)
    self.pool3 = nn.MaxPool2d(3, stride=2)
    self.fc1 = nn.Linear(256 * 6 * 6, 2056)
    self.fc2 = nn.Linear(2056,512)
    self.fc3 = nn.Linear(512,256)
    self.fc4 = nn.Linear(256,64)
    self.fc5 = nn.Linear(64,6)

  def forward(self, x):
    x = self.pool1(F.relu(self.conv1(x)))
    x = self.pool2(F.relu(self.conv2(x)))
    x = self.pool3(F.relu(self.conv3(x)))
    x = x.view(-1, 256 * 6 * 6)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))
    x = F.relu(self.fc4(x))
    x = self.fc5(x)
    return x

In [ ]:
classify_net = Net()

In [ ]:
classify_net.load_state_dict(torch.load("/content/drive/MyDrive/Fall 2022/ECS271/detecto/custom_classificaton_model.pt"))

In [ ]:
classify_net.eval()

In [ ]:
classes = {'billete':0, 'knife':1, 'monedero':2, 'pistol':3, 'smartphone':4, 'tarjeta':5}

In [ ]:
import os
import xml.etree.ElementTree as ET

y_true = []
y_pred = []
count = 0

directory = '/content/drive/MyDrive/Fall 2022/ECS271/detecto/Sohas_weapon-Detection/test'

for filename in os.listdir(directory):
  if filename.endswith(".jpg") and os.path.isfile(os.path.join(directory, filename.split('.')[0] + ".xml")):
    f = os.path.join(directory, filename)

    if os.path.isfile(f):
      # passing the image through Detecto model
      image = utils.read_image(f)
      predictions = model.predict(image)
      labels, boxes, scores = predictions

      if scores.size()[0] == 0:
        continue

      count += 1
      print("Testing image " + str(count))
      
      x = int(boxes[np.argmax(scores)][0])
      y = int(boxes[np.argmax(scores)][1])
      w = int(boxes[np.argmax(scores)][2])
      h = int(boxes[np.argmax(scores)][3])
      img = cv2.imread(f)
      cropped_img = img[y:h, x:w]
      
      tfms = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(256),
        transforms.CenterCrop(225),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

      norm_image = tfms(cropped_img).to('cpu').unsqueeze(0)
      output = classify_net(norm_image)
      output = torch.tensor(output, requires_grad=True)
      output = output.detach().numpy()
      y_pred.append(np.argmax(output))

      # extracting ground truth
      tree = ET.parse(os.path.join(directory, filename.split('.')[0] + ".xml"))
      root = tree.getroot()
      objects = root.findall('object')
      for obj in objects:
        y_true.append(classes[obj.find('name').text])
      


In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import seaborn as sns

In [ ]:
sns.heatmap(confusion_matrix(y_true, y_pred), annot=True, cmap='Blues')

In [ ]:
target_classes = ['billete', 'knife', 'monedero', 'pistol', 'smartphone', 'tarjeta']
print(classification_report(y_true, y_pred, target_names=target_classes))